In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import geopandas

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
df = pd.read_csv('polio.csv')    # http://polioeradication.org/where-we-work/polio-free-countries/

In [2]:
dic = {'Bosnia and Herzegovina':'Bosnia and Herz.', 
       'Brunei Darussalam':'Brunei', 
       'Central African Republic':'Central African Rep.',
       "Cote d’Ivoire":"Côte d'Ivoire",
       'Democratic Republic of the Congo':'Dem. Rep. Congo',
       'Czech Republic':'Czech Rep.',
       'Dominican Republic':'Dominican Rep.',
       'Equatorial Guinea':'Eq. Guinea',
       'Democratic People’s Rep. of Korea':'Dem. Rep. Korea',
       'Republic of Korea':'Korea',
       'Lao People’s Democratic Republic':'Lao PDR',
       'Libyan Arab Jamahiriya':'Libya',
       'Maldives':'Falkland Is.',
       'Former Yugoslav Rep. of Macedonia':'Macedonia',
       'Republic of Moldova':'Moldova',
       'Palestine N.A.':'Palestine',
       'Russian Federation':'Russia',
       'Syrian Arab Republic':'Syria',
       'Solomon Islands':'Solomon Is.',
       'United Republic of Tanzania':'Tanzania',
       'United States of America':'United States',
       'Venezuala':'Venezuela',
       'Viet Nam':'Vietnam',
       'East Timor':'Timor-Leste',
       
      }
for key, value in dic.items():
    i = df[df['Country']==key].Country.index
    df.at[i,'Country'] = value

In [3]:
# Add territories that were listed as one 

# Kosovo, Serbia and Montenegro were not listed in WHO list. 
# Since they are considered polio free but there is no record of date, I'll consider it's the same as Macedonia (1993)
# Croatia is in the same region and got polio free by 1990, I think this should be close to the date of it's neighboors

dic = { 'S. Sudan' : 'Sudan',
        'Somaliland' : 'Somalia',
        'Kosovo' : 'Macedonia',    
        'Serbia' : 'Macedonia',
        'Montenegro' : 'Macedonia',
        'N. Cyprus' : 'Cyprus',
        'Greenland' : 'Denmark'}

for key, value in dic.items():
    dic[key] = int(df[df['Country'] == value]['Year'].values[:1])
    
# Add values that wasn't listed

# W. Sahara - I couldn't find information about Western Sahara 
# I could use information from Morroco, but prefered to leave it as unavailable

# Taiwan - https://www.mohw.gov.tw/cp-116-256-2.html
new_dic = {'W. Sahara':'data not available', 'Taiwan':2000}
dic.update(new_dic)

# create df from dicts
df_extenssion = pd.DataFrame.from_dict(dic, orient = 'index', columns = ['Year'])
df_extenssion

,Year
S. Sudan,2004
Somaliland,2002
Kosovo,1993
Serbia,1993
Montenegro,1993
N. Cyprus,1995
Greenland,1976
W. Sahara,data not available
Taiwan,2000


In [4]:
# set df index to country column and append new values to df
df = df.set_index('Country')
df = df.append(df_extenssion)

In [5]:
# Check values in map and not in the data

#for i, row in world.iterrows():
#    if row['name'] not in df.index:
#        print(row['name'])

In [6]:
# Check values in the data and not in maps

#for i, row in df.iterrows():
#    if row.Country not in world.name.values:
#        print(row.Country)

In [7]:
# I'll loop the years and display the countries that were already considered polio-free by those years
# So I used an unreacheable year for the countries with status 'ongoing'
# and set the unavailable data to None 
for i, row in df.iterrows():
    if(row.Year == 'data not available'):
        row.Year = None
    elif(row.Year == 'ongoing'):
        row.Year = 9999

In [8]:
# Fix df for plotting
df = pd.to_numeric(df['Year']) # build series with year values as numeric
df = pd.DataFrame(df) # rebuild df from series
df.dropna(inplace = True) # drop the 'data not available' countries
df.reset_index(inplace = True) # reset index
df.columns = ['Country', 'Year'] # set column names

In [9]:
# define a function that return a copy of the dataframe with records that with a year lower than the passed
def get_df(df, year):
    df_year = df[df['Year'] <= year].copy()
    return df_year

In [11]:
# get all countries with a numeric year and merge with the world map
all_data = world.merge(get_df(df, 9999), how='left', left_on = 'name', right_on = 'Country')
# build a custom colormap
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#edf8b1","#6DC661"])
# build separate legend handlers
legend_elements = [Patch(facecolor='#6DC661', edgecolor='grey', label='Color Patch'),
                  Patch(facecolor='#edf8b1', edgecolor='grey', label='Color Patch')]

years = np.arange(1970,2019)
# Plot maps 
for year in years:
    # merge world map with filtered df into a new dataframe
    temp = world.merge(get_df(df, year), how='left', left_on = 'name', right_on = 'Country')
    
    fig, ax = plt.subplots(1, figsize = (40, 24))
    fig.patch.set_facecolor('#BAE3FF')
    # plot colormap
    temp.plot(column='Year', cmap=cmap, vmin=-1, vmax=1, ax = ax, edgecolor = '0.3')
    # plot unlisted countries above
    all_data[all_data['Year'].isna()].plot(color = 'white', ax = ax, edgecolor = '0.3')
    
    # remove axis and plot title
    ax.axis('off')
    plt.title('POLIO-FREE COUNTRIES\n' + str(year), fontsize=32)
    # plot info on the bottom
    info = 'Data from last recorded case of indigenous wild poliovirus (WPV)\nsource: Global Polio Eradication Initiative'
    plt.text(0,-110, info, ha = 'center', fontsize = 16)
    # plot legend
    ax.legend(handles=legend_elements,labels = ['Polio-Free', 'Ongoing'], loc='center', 
          fontsize = 20, bbox_to_anchor=(0.5, 0), ncol=4)
    
    # Save pic and close plot
    plt.savefig(str(year) + '.png', facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close()
    #plt.show()

In [12]:
# build video
import moviepy.editor as mp

clip_list = [str(i) + '.png' for i in years]
clip = mp.ImageSequenceClip(clip_list, fps=2)
clip.write_videofile("myvideo.mp4")

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Moviepy - Building video myvideo.mp4.
Moviepy - Writing video myvideo.mp4



Moviepy - Done !
Moviepy - video ready myvideo.mp4
